# Creating a Operator

In [1]:
import sys
sys.path.insert(0, '../') 
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()
%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Use the different modules to create an operator instance
from openoperator import OpenOperator
from openoperator.blob_store import AzureBlobStore
from openoperator.document_loader import UnstructuredDocumentLoader
from openoperator.vector_store import PGVectorStore
from openoperator.embeddings import OpenAIEmbeddings
from openoperator.knowledge_graph import KnowledgeGraph
from openoperator.llm import OpenAILLM

# Create the different modules that are needed for the operator
blob_store = AzureBlobStore()
document_loader = UnstructuredDocumentLoader()
embeddings = OpenAIEmbeddings()
vector_store = PGVectorStore(embeddings=embeddings)
knowledge_graph = KnowledgeGraph()
llm = OpenAILLM(model_name="gpt-4-0125-preview")

operator = OpenOperator(
    blob_store=blob_store,
    document_loader=document_loader,
    vector_store=vector_store,
    embeddings=embeddings,
    knowledge_graph=knowledge_graph,
    llm=llm
)

## Create a Portfolio and a Facility

In [3]:
portfolio = operator.create_portfolio('Setty & Associates LLC 2')
facility = portfolio.create_facility('DC Office')

## Lets load all the files for the facility

In [4]:
import mimetypes
import os
directory_path = "setty_office_data/documents/" 

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        file_type = mimetypes.guess_type(file_path)[0]
        with open(file_path, 'rb') as file:
            file_content = file.read()
            facility.upload_document(file_content, filename, file_type=file_type)

## Now we can search through all the data within those documents

In [5]:
# Search for metadata in documents
docs = facility.search_documents({
    "query": "AHU",
    "limit": 2
})
docs

[{'content': 'AHU SCHEDULE\n\nOmniClass Number\n\n23.33.39.17.13.11 23.33.39.17.13.11\n\nOmniClass Title\n\nAir Cooled Single Pack Packaged Air Conditioners Air Cooled Single Pack Packaged Air Conditioners\n\nMark\n\nAHU-1 AHU-2\n\nFuel Type\n\nNATURAL GAS NATURAL GAS\n\nInternal Efficiency (%) 82 82\n\nMax External Static Pressure 1" W.C 1" W.C\n\nCFM (nominal) Heating Input Max.\n\n0 CFM 0 CFM\n\n224000 BTU/Hr 224000 BTU/Hr\n\ncooling\n\nHeating Output Capacity\n\n140000 BTU/Hr 184000 BTU/Hr 140000 BTU/Hr 184000 BTU/Hr\n\nEER\n\n10.8 10.8\n\nCOMPRESSOR A LRA 123 123',
  'metadata': {'file_url': 'https://syystorage.blob.core.windows.net/open-operator-2/2023-12-30_DC%20Office_Mechanical.pdf',
   'filename': '2023-12-30_DC Office_Mechanical.pdf',
   'filetype': 'application/pdf',
   'languages': ['eng'],
   'page_number': 7,
   'facility_uri': 'https://openoperator.com/setty%20%26%20associates/dc%20office',
   'portfolio_uri': 'https://openoperator.com/setty%20%26%20associates'}},
 {'co

## Upload COBie spreadsheets

In [ ]:
import os
directory_path = "setty_office_data/cobie_sheets/"

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'rb') as file:
            file_content = file.read()
            errors_found, errors = facility.upload_cobie_spreadsheet(file_content)
            if errors_found:
                print(f"Errors found in {filename}: {errors}")

## Upload Bacnet data 

In [4]:
with open("../data/dunbar/dunbar_bacnet.json", 'rb') as file:
    file_content = file.read()
    facility.bas.upload_bacnet_data(file_content)

In [14]:
facility.bas.points()

[{'bacnet__object_index': '16',
  'bacnet__scrape_interval': '0',
  'bacnet__object_name': 'SCH16',
  'bacnet__scrape_enabled': 'False',
  'bacnet__device_id': '88004',
  'bacnet__present_value': 'Null',
  'bacnet__device_description': '',
  'bacnet__device_name': 'pCOWeb88004',
  'bacnet__device_address': '192.168.1.11',
  'bacnet__object_units': '',
  'bacnet__object_description': 'Schedule 16',
  'uri': 'https://openoperator.com/setty%20%26%20associates%20llc%202/dc%20office/device/192.168.1.11-88004/pcoweb88004/point/schedule/sch16/16'},
 {'bacnet__object_index': '4011',
  'bacnet__raw_object_type': 'trendLog',
  'bacnet__object_name': 'CWP-P3-START-RTL',
  'bacnet__device_id': '1000',
  'bacnet__raw_index': '4011',
  'bacnet__raw_objectName': 'CWP-P3-START-RTL',
  'uri': 'https://openoperator.com/setty%20%26%20associates%20llc%202/dc%20office/device/192.168.1.1-1000/chilled/cond%20water/point/trendLog/cwp-p3-start-rtl/4011',
  'bacnet__scrape_interval': '0',
  'bacnet__scrape_enab

In [36]:
facility.bas.vectorize_graph()

token count
2006
finished chunk
token count
81850
Token count is greater than 8000, splitting the texts into chunks.
finished chunk
finished chunk
finished chunk
finished chunk
finished chunk
finished chunk
finished chunk
finished chunk
finished chunk
finished chunk
finished chunk
14340
14340


In [41]:
facility.bas.cluster_devices()

Clustering...
Estimated number of clusters: 12
Estimated number of noise points: 31


{0: ['Eaton Drive-5',
  'Eaton Drive-4',
  'Eaton Drive-7',
  'Eaton Drive-11',
  'Eaton Drive-3',
  'Eaton Drive-1',
  'Eaton Drive-31',
  'Eaton Drive-16',
  'Eaton Drive-17',
  'Eaton Drive-14',
  'Eaton Drive-19',
  'Eaton Drive-14',
  'Eaton Drive-18',
  'Eaton Drive-13'],
 -1: ['MTIIChillerUC100001',
  'CHILLED/COND WATER',
  'enteliBUS Manager 4104766',
  'pCOWeb88004',
  'A-WING FLRS 3&4 DOAS2',
  'HOT WATER PLANT',
  'Volttron BACnet driver',
  'Device_0030de05fef2',
  'B-WING VAVS',
  'MTIIChillerUC200002',
  'C-WING RTU DOAS1&6',
  "DOAS-3&5 VAV'S",
  'A-WING FLOORS 1&2',
  'FCU-14&15 2ND FLR',
  'GARAGE EXHAUST',
  'EF-18',
  'FCU-11&12 1ST FLR',
  'pCOnet77001',
  'FCU-13',
  'EF-17',
  'EF-23',
  'Dev_MSTP',
  'pCOnet77006',
  'DHW',
  'WATT_BACnet_66005',
  'EF-5&9',
  'WATT_BACnet',
  'WSHP-24',
  'WSHP-19',
  'WSHO-20',
  'WSHP-18'],
 1: ['Engineering', 'Engineering', 'Engineering'],
 2: ['BTU METER 1020', 'BTU METER 1022', 'BTU METER 1021', 'BTU METER 2020'],
 3: ['AT

In [42]:
facility.bas.cluster_points()

Clustering...
Estimated number of clusters: 36
Estimated number of noise points: 51


{0: ['SCH16',
  'CWP-P3-START-RTL',
  'CHW-ENABLE-RTL',
  'CHWP-P11-AMPS',
  'CHWP-P11-SPEED',
  'Fourth Floor Average Zone Temp PG',
  'A167',
  'D108',
  'GEO-VAULT-REQUEST',
  'Max Frequency',
  'CAL3',
  'C-SEC-CWP-LOOP',
  'PUMP-P4-S-RTL',
  'Warning State',
  'Energy Rate',
  'I054',
  'RUN/STOP ACT',
  'A207',
  'Chilled Water Pumps Running',
  'D067',
  'D189',
  'NC7',
  'SurfTemp_F',
  'Motor Speed',
  'D079',
  'McQuayChillerType',
  'General 0',
  'I079',
  'Fault State',
  'Cmpr1Curr',
  'PUMP-P10-S-RTL',
  'A176',
  'D117',
  'AirOffEvapTemp',
  'CHWP-P10-START-RTL',
  'LiquidLineTemp',
  'General 4',
  'D196',
  'Current',
  'A085',
  'PurgeModeActv',
  'D080',
  'POOL-PAK-RARH-SP',
  'General 2',
  'A158',
  'A102',
  'General 6',
  'D140',
  'Motor Current',
  'OA-ENTHALPY-SP',
  'Output Frequency',
  'Mode 2 Energy Total',
  'AIProblemAlarm',
  'Active Fault Code',
  'DOAS DR Shutdown',
  'A053',
  'CompSelect',
  'Mode 1 Energy Total',
  'NC10',
  'CHWR-TEMP',
  'I14

## Chat with the AI Assistant

In [19]:
messages = [
    {
        "role": "user",
        "content": "create a table of my air handling units schedule"
    }
]
for response in operator.chat(messages, portfolio, facility, verbose=True):
    print(response, flush=True, end="")

Tool Selected: search_building_documents
Tool args: {'query': 'air handling units schedule'}
Tool response:
[
  {
    "content": "AIR DEVICE SCHEDULE\n\nOmniClass Number\n\n23.33.43.15\n\n23.33.43.15\n\n23.33.43.15\n\nOmniClass Title Refrigeration Condenser Units Refrigeration Condenser Units Refrigeration Condenser Units\n\nMark\n\nACCU-1\n\nACCU-2\n\nACCU-3\n\nManufacturer\n\nMitsubishi Electric\n\nMitsubishi Electric\n\nMitsubishi Electric\n\nOmniClass Number\n\n23.33.39.21\n\n23.33.39.21\n\n23.33.39.21\n\nOmniClass Title Split System Air Conditioning Units Split System Air Conditioning Units Split System Air Conditioning Units\n\nMark\n\nManufacturer\n\nAC-1 Mitsubishi Electric",
    "metadata": {
      "file_url": "https://syystorage.blob.core.windows.net/open-operator-2/2023-12-30_DC%20Office_Mechanical.pdf",
      "filename": "2023-12-30_DC Office_Mechanical.pdf",
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "page_number": 7,
      "faci